In [27]:
pwd

'/home/jupyter/Summer_AMPL/Renate/Kevin_Model_2'

In [15]:
cd /home/jupyter/Summer_AMPL/Renate

/home/jupyter/Summer_AMPL/Renate


In [30]:
cd /home/jupyter/Summer_AMPL/Renate/QM9_Curation

/home/jupyter/Summer_AMPL/Renate/QM9_Curation


In [31]:
pwd

'/home/jupyter/Summer_AMPL/Renate/QM9_Curation'

In [32]:
!tar -tf *.tar.gz

tar: select_qm9_data_curated_model_6b863442-a01d-4f8d-a622-9364e7c82b61.tar.gz: Not found in archive
tar: select_qm9_data_curated_model_71397a70-cef7-4d04-822d-5828768a2f62.tar.gz: Not found in archive
tar: select_qm9_data_curated_model_89cd7a1f-6531-454c-bc1f-392cff327831.tar.gz: Not found in archive
tar: select_qm9_data_curated_model_9cec60e2-8d4f-4a01-bb1c-0032313f2277.tar.gz: Not found in archive
tar: select_qm9_data_curated_model_a14a4caa-aa6f-4dfd-a892-374a1fc19ee4.tar.gz: Not found in archive
tar: select_qm9_data_curated_model_cd65e84f-2498-436b-ac43-13b17e90e48c.tar.gz: Not found in archive
tar: Exiting with failure status due to previous errors


In [19]:
!tar -xvzf *.tar.gz

./
./best_model/
./best_model/checkpoint
./best_model/ckpt-18.data-00000-of-00001
./best_model/ckpt-18.index
./model_metadata.json
./model_metrics.json
./transformers.pkl


In [20]:
#!cat FMO_energy_graphconv_fingerprint_split_model_d852304e-cd01-43b1-91ca-15125b225fc6/model_metadata.json| head
!cat Kevin_Moel/model-metadata.json| head

{
    "model_parameters": {
        "ampl_version": "1.5.0",
        "featurizer": "graphconv",
        "hyperparam_uuid": null,
        "model_bucket": "public",
        "model_choice_score_type": "r2",
        "model_type": "NN",
        "num_model_tasks": 3,
        "prediction_type": "regression",


In [21]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_context("poster")
sns.set_style("whitegrid")
sns.set_palette("Set2")

import pandas as pd
import os, json, sys, glob, pickle

In [22]:
from atomsci.ddm.pipeline import parameter_parser as parse
from atomsci.ddm.pipeline import perf_data
from atomsci.ddm.pipeline import model_pipeline as mp
import atomsci.ddm.utils.curate_data as curate_data
import atomsci.ddm.utils.struct_utils as struct_utils
from atomsci.ddm.pipeline import perf_plots as pp

/home/jupyter/atomsci/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/jupyter/atomsci/lib/python3.8/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.
2023-07-06 15:30:27,181 Model tracker client not supported in your environment; will save models in filesy

In [23]:
test_data = pd.read_csv('select_qm9_data_curated-Copy1.csv')

In [24]:
test_data.head()

,mol_id,smiles,homo,lumo,gap,rdkit_smiles,inchi_key,compound_id,mol_wt,VALUE_NUM_mean,VALUE_NUM_std,Perc_Var,Remove_BadDuplicate
0,gdb_1,C,-0.3877,0.1171,0.5048,C,VNWKTOKETHGBQD-UHFFFAOYSA-N,VNWKTOKETHGBQD-UHFFFAOYSA-N,16.031300,0.5048,NaN,0.0,0
1,gdb_2,N,-0.2570,0.0829,0.3399,N,QGZKDVFQNNGYKY-UHFFFAOYSA-N,QGZKDVFQNNGYKY-UHFFFAOYSA-N,17.026549,0.3399,NaN,0.0,0
2,gdb_3,O,-0.2928,0.0687,0.3615,O,XLYOFNOQVPJJNP-UHFFFAOYSA-N,XLYOFNOQVPJJNP-UHFFFAOYSA-N,18.010565,0.3615,NaN,0.0,0
3,gdb_4,C#C,-0.2845,0.0506,0.3351,C#C,HSFWRNGVRCDJHI-UHFFFAOYSA-N,HSFWRNGVRCDJHI-UHFFFAOYSA-N,26.015650,0.3351,NaN,0.0,0
4,gdb_5,C#N,-0.3604,0.0191,0.3796,C#N,LELOWRISYMNNSU-UHFFFAOYSA-N,LELOWRISYMNNSU-UHFFFAOYSA-N,27.010899,0.3796,NaN,0.0,0


In [10]:
test_data['rdkit_smiles'] = test_data['smiles'].apply(curate_data.base_smiles_from_smiles)
test_data['inchi_key'] = test_data['rdkit_smiles'].apply(struct_utils.smiles_to_inchi_key)
test_data['compound_id'] = test_data['inchi_key']

In [11]:
test_data.head()

,compound,smiles,rdkit_smiles,inchi_key,compound_id
0,file gdb 133885,[C@]123[C@@H]4[C@H]5[C@H]([C@H]5O3)[C@H]1N4C2,C1N2[C@H]3[C@@H]4[C@@H]5O[C@]13[C@H]2[C@@H]54,AZDPPUQTZXTYOS-GWJPIIGYSA-N,AZDPPUQTZXTYOS-GWJPIIGYSA-N


In [25]:
import os
files = os.listdir()
for f in files: 
  if (f.endswith(".tar.gz")):
    mfile = f 

print(mfile)

FMO_energy_graphconv_fingerprint_split_model_d852304e-cd01-43b1-91ca-15125b225fc6.tar.gz


In [28]:
from atomsci.ddm.pipeline import predict_from_model as pfm
mfile = "FMO_energy_graphconv_fingerprint_split_model_d852304e-cd01-43b1-91ca-15125b225fc6.tar.gz"
mpath = '/content'
input_df = test_data
response_col = "HOMO_energy,LUMO_energy,HOMO_LUMO_gap"
compound_id = 'compound_id'
smiles_col = "rdkit_smiles"
results_df = pfm.predict_from_model_file(model_path = mfile, 
                            input_df = input_df, 
                            smiles_col = smiles_col, 
                            response_col = response_col)

Standardizing SMILES strings for 129435 compounds.


2023-07-06 16:05:29,068 Aborting reionization to avoid infinite loop due to it being ambiguous where to put a Hydrogen
2023-07-06 16:05:31,630 Aborting reionization to avoid infinite loop due to it being ambiguous where to put a Hydrogen
2023-07-06 16:05:38,104 Aborting reionization to avoid infinite loop due to it being ambiguous where to put a Hydrogen
2023-07-06 16:05:55,514 Aborting reionization to avoid infinite loop due to it being ambiguous where to put a Hydrogen
2023-07-06 16:06:01,553 Aborting reionization to avoid infinite loop due to it being ambiguous where to put a Hydrogen
2023-07-06 16:06:04,351 Aborting reionization to avoid infinite loop due to it being ambiguous where to put a Hydrogen
2023-07-06 16:06:08,260 Aborting reionization to avoid infinite loop due to it being ambiguous where to put a Hydrogen
2023-07-06 16:06:08,313 Aborting reionization to avoid infinite loop due to it being ambiguous where to put a Hydrogen
2023-07-06 16:06:17,389 Aborting reionization to

num_model_tasks is deprecated and its value is ignored.


ValueError: in user code:

    File "/home/jupyter/atomsci/lib/python3.8/site-packages/deepchem/models/keras_model.py", line 698, in _compute_model  *
        return self.model(inputs, training=False)
    File "/home/jupyter/atomsci/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/var/tmp/__autograph_generated_filegp3xpraw.py", line 67, in tf__call
        ag__.for_stmt(ag__.converted_call(ag__.ld(range), (ag__.converted_call(ag__.ld(len), (ag__.ld(self).graph_convs,), None, fscope),), None, fscope), None, loop_body, get_state_2, set_state_2, ('in_layer',), {'iterate_names': 'i'})
    File "/var/tmp/__autograph_generated_filegp3xpraw.py", line 28, in loop_body
        gc1 = ag__.converted_call(ag__.ld(self).graph_convs[ag__.ld(i)], (ag__.ld(gc_in),), None, fscope)
    File "/home/jupyter/atomsci/lib/python3.8/site-packages/deepchem/models/layers.py", line 129, in build
        sel

In [13]:
results_df.head()

NameError: name 'results_df' is not defined

In [20]:
print (results_df)

          compound                                         smiles  \
0  file gdb 133885  [C@]123[C@@H]4[C@H]5[C@H]([C@H]5O3)[C@H]1N4C2   

                                   rdkit_smiles                    inchi_key  \
0  C1N2[C@H]3[C@H]4[C@@H]5[C@H]4O[C@]13[C@@H]52  AZDPPUQTZXTYOS-GWJPIIGYSA-N   

                   compound_id                                   orig_smiles  \
0  AZDPPUQTZXTYOS-GWJPIIGYSA-N  C1N2[C@H]3[C@H]4[C@@H]5[C@H]4O[C@]13[C@@H]52   

   VALUE_NUM_mean_pred  
0             0.416887  
